# Circuit simulation examples

In [1]:
import obi_one as obi

In [2]:
# circuit_path_prefix = "/Users/james/Documents/obi/additional_data/"
circuit_path_prefix = "/Users/armando/Desktop/Models/SSCx/O1_data/"
# circuit_path_prefix = "/Users/pokorny/Data/Circuits/"

### Loading two circuits

In [3]:
# circuit = obi.Circuit(name="ToyCircuit-S1-6k", path=circuit_path_prefix + "ToyCircuit-S1-6k/circuit_config.json")
circuit = obi.Circuit(name="O1", path=circuit_path_prefix + "circuit_config.json")
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")

Circuit 'O1' with 785632 neurons and 560631825 synapses


In [4]:
# circuit2 = obi.Circuit(name="ToyCircuit-S1-6k-duplicate", path="/Users/pokorny/Data/Circuits/ToyCircuit-S1-6k/circuit_config.json")
# circuit2 = obi.Circuit(name="nbS1-HEX0-beta", path=circuit_path_prefix + "ToyCircuit-S1-6k/circuit_config.json")
circuit2 = obi.Circuit(name="O1_2", path=circuit_path_prefix + "circuit_config.json")
print(f"Circuit '{circuit2}' with {circuit2.sonata_circuit.nodes.size} neurons and {circuit2.sonata_circuit.edges.size} synapses")

Circuit 'O1_2' with 785632 neurons and 560631825 synapses


### Defining the circuit simulation

In [5]:
# Simulation init
sim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1")
sim_duration = 3000 # ms
poisson_stim_duration = 800 # ms
poisson_stim_frequency = 20 # Hz
simulations_initialize = obi.SimulationsForm.Initialize(
    circuit=[circuit, circuit2],
    node_set=sim_neuron_set,
    simulation_length=sim_duration,
)

# Stimuli
stim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", random_sample=[10, 20])
stim_times = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=1000.0)  # in ms!!
current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=stim_times, duration=5.0, neuron_set=stim_neuron_set, amplitude=[0.2, 0.5])
sync_current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=stim_times, duration=100.0, neuron_set=stim_neuron_set, amplitude=0.1)
poisson_input = obi.PoissonSpikeStimulus(timestamps=stim_times,
                                                                        stim_duration=poisson_stim_duration,
                                                                        frequency=poisson_stim_frequency,
                                                                        neuron_set=stim_neuron_set,
                                                                       )

# Recordings
rec_neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", random_sample=100)
v_recording = obi.SomaVoltageRecording(start_time=0.0, end_time=sim_duration, neuron_set=rec_neuron_set)

"""
Fill form with Blocks
"""
simulations_form = obi.SimulationsForm(
                                    initialize=simulations_initialize,
                                    intracellular_location_sets={},
                                    extracellular_location_sets={},
                                    neuron_sets={"L1All": sim_neuron_set, "L1Stim": stim_neuron_set, "L1Rec": rec_neuron_set},
                                    synapse_sets={},
                                    timestamps={"StimTimes": stim_times},
                                    stimuli={"CurrentStimulus": current_stimulus, "SyncCurrentStimulus": sync_current_stimulus, "PoissonInputStimulus": poisson_input},
                                    recordings={"SomaVoltRec": v_recording},
                                    )

In [6]:
simulations_form.model_dump(mode="json")

{'type': 'SimulationsForm',
 'timestamps': {'StimTimes': {'type': 'RegularTimestamps',
   'start_time': 0.0,
   'simulation_level_name': 'StimTimes',
   'number_of_repetitions': 3,
   'interval': 1000.0}},
 'stimuli': {'CurrentStimulus': {'type': 'ConstantCurrentClampSomaticStimulus',
   'timestamps': {'type': 'RegularTimestamps',
    'start_time': 0.0,
    'simulation_level_name': 'StimTimes',
    'number_of_repetitions': 3,
    'interval': 1000.0},
   'simulation_level_name': 'CurrentStimulus',
   'delay': 0.0,
   'duration': 5.0,
   'neuron_set': {'type': 'PredefinedNeuronSet',
    'simulation_level_name': 'L1Stim',
    'random_sample': [10, 20],
    'random_seed': 0,
    'node_set': 'Layer1'},
   'represents_physical_electrode': False,
   'amplitude': [0.2, 0.5]},
  'SyncCurrentStimulus': {'type': 'ConstantCurrentClampSomaticStimulus',
   'timestamps': {'type': 'RegularTimestamps',
    'start_time': 0.0,
    'simulation_level_name': 'StimTimes',
    'number_of_repetitions': 3,
    

In [7]:
grid_scan = obi.GridScan(form=simulations_form, output_root='../../obi-output/circuit_simulations/grid_scan')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute(processing_method='generate')


MULTIPLE VALUE PARAMETERS
stimuli.CurrentStimulus.amplitude: [0.2, 0.5]
neuron_sets.L1Stim.random_sample: [10, 20]
initialize.circuit: [Circuit(type='Circuit', name='O1', path='/Users/armando/Desktop/Models/SSCx/O1_data/circuit_config.json'), Circuit(type='Circuit', name='O1_2', path='/Users/armando/Desktop/Models/SSCx/O1_data/circuit_config.json')]

COORDINATE PARAMETERS
stimuli.CurrentStimulus.amplitude: 0.2, neuron_sets.L1Stim.random_sample: 10, initialize.circuit: O1
stimuli.CurrentStimulus.amplitude: 0.2, neuron_sets.L1Stim.random_sample: 10, initialize.circuit: O1_2
stimuli.CurrentStimulus.amplitude: 0.2, neuron_sets.L1Stim.random_sample: 20, initialize.circuit: O1
stimuli.CurrentStimulus.amplitude: 0.2, neuron_sets.L1Stim.random_sample: 20, initialize.circuit: O1_2
stimuli.CurrentStimulus.amplitude: 0.5, neuron_sets.L1Stim.random_sample: 10, initialize.circuit: O1
stimuli.CurrentStimulus.amplitude: 0.5, neuron_sets.L1Stim.random_sample: 10, initialize.circuit: O1_2
stimuli.Curr

AssertionError: Output file '../../obi-output/circuit_simulations/grid_scan/stimuli.CurrentStimulus.amplitude=0.2/neuron_sets.L1Stim.random_sample=10/initialize.circuit=O1/node_sets.json' already exists! Delete file or choose to overwrite.

In [ ]:
# Deserialization
# grid_scan_ds = obi.deserialize_obi_object_from_json_file("../../obi-output/circuit_simulations/grid_scan/run_scan_config.json")